In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil import parser
import re
import matplotlib.patches as patches
import matplotlib
!pip install inflect
import inflect

In [2]:
data = pd.read_csv("Tarkari.csv", low_memory=False)
data.head()

,Commodity,Date,Unit,Minimum,Maximum,Average
0,Tomato Big(Nepali),6/16/2013,Kg,35,40,37.5
1,Tomato Small(Local),6/16/2013,Kg,26,32,29
2,Potato Red,6/16/2013,Kg,20,21,20.5
3,Potato White,6/16/2013,Kg,15,16,15.5
4,Onion Dry (Indian),6/16/2013,Kg,28,30,29


In [3]:
data['Date'] = pd.to_datetime(data['Date'])

In [4]:
pattern = r"^Rs\s+"
data["Minimum"] = pd.to_numeric(data["Minimum"].apply(lambda x: x if not str(x).startswith("Rs") else re.sub(pattern,"",str(x))))
data["Maximum"] = pd.to_numeric(data["Maximum"].apply(lambda x: x if not str(x).startswith("Rs") else re.sub(r"^Rs\s+", "",str(x))))
data["Average"] = pd.to_numeric(data["Average"].apply(lambda x: x if not str(x).startswith("Rs") else re.sub(r"^Rs\s+", "", str(x))))

In [5]:
selected_commodities = ['Onion Dry (Indian)', 'Tomato Big(Nepali)', 'Potato White', 'Carrot(Local)', 'Apple(Jholey)', 'Spinach', 'Banana', 'Cucumber(Local)', 'Garlic Dry Nepali', 'Lettuce']
selected_data = data[data['Commodity'].isin(selected_commodities)].copy()

In [6]:
selected_data.loc[:, 'Season'] = selected_data['Date'].dt.month.map({1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'})

In [7]:
selected_data.head()

,Commodity,Date,Unit,Minimum,Maximum,Average,Season
0,Tomato Big(Nepali),2013-06-16,Kg,35.0,40.0,37.5,Summer
3,Potato White,2013-06-16,Kg,15.0,16.0,15.5,Summer
4,Onion Dry (Indian),2013-06-16,Kg,28.0,30.0,29.0,Summer
5,Carrot(Local),2013-06-16,Kg,30.0,35.0,32.5,Summer
40,Lettuce,2013-06-16,Kg,55.0,60.0,57.5,Summer


In [9]:
selected_data['Date'] = pd.to_datetime(selected_data['Date'])

# Extract features from the date
selected_data['day'] = selected_data['Date'].dt.day
selected_data['month'] = selected_data['Date'].dt.month
selected_data['year'] = selected_data['Date'].dt.year


In [10]:
selected_data['Season'] = pd.Categorical(selected_data['Season'])

# Create one-hot encoded columns
season_dummies = pd.get_dummies(selected_data['Season'], prefix='Season')

# Concatenate the one-hot encoded columns with the original DataFrame
selected_data = pd.concat([selected_data, season_dummies], axis=1)

# Drop the original 'season' column if needed
selected_data.drop('Season', axis=1, inplace=True)

# Display the updated DataFrame
print(selected_data.head())

             Commodity       Date Unit  Minimum  Maximum  Average  day  month  \
0   Tomato Big(Nepali) 2013-06-16   Kg     35.0     40.0     37.5   16      6   
3         Potato White 2013-06-16   Kg     15.0     16.0     15.5   16      6   
4   Onion Dry (Indian) 2013-06-16   Kg     28.0     30.0     29.0   16      6   
5        Carrot(Local) 2013-06-16   Kg     30.0     35.0     32.5   16      6   
40             Lettuce 2013-06-16   Kg     55.0     60.0     57.5   16      6   

    year  Season_Fall  Season_Spring  Season_Summer  Season_Winter  
0   2013        False          False           True          False  
3   2013        False          False           True          False  
4   2013        False          False           True          False  
5   2013        False          False           True          False  
40  2013        False          False           True          False  


In [11]:
selected_data.to_csv('C:/Users/nirvi/OneDrive/Desktop/Programs/Mlcurrent/features.csv', index=False)